In [1]:
from model import river_model, losses, dot_prod_attention
from data import data_generation, batch_creator, gp_kernels
from keras.callbacks import ModelCheckpoint
from helpers import helpers, masks, metrics
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from inference import infer
import tensorflow as tf
import numpy as np
import matplotlib 
import time
import keras
plt.style.use('ggplot')

Using TensorFlow backend.


In [2]:
import pandas as pd

In [3]:
attributes = pd.read_csv('/Users/omernivron/Downloads/attibutes.csv')

In [4]:
attributes_numeric = attributes[['gauge_id', 
               'p_mean', 'pet_mean', 'p_seasonality', 'frac_snow',
               'aridity', 'high_prec_freq', 'high_prec_dur',
                #'high_prec_timing',
               'low_prec_freq', 'low_prec_dur',
                #'low_prec_timing', 'geol_1st_class', 'glim_1st_class_frac', 'geol_2nd_class', 'glim_2nd_class_frac',
               'carbonate_rocks_frac', #'geol_porostiy',
                'geol_permeability',
                #'q_mean','runoff_ratio', 'slope_fdc', 'baseflow_index', 'stream_elas', 'q5',
               #'q95', 'high_q_freq', 'high_q_dur', 'low_q_freq', 'low_q_dur',
               #'zero_q_freq', 'hfd_mean', 'huc_02', 'gauge_name',
               'soil_depth_pelletier', 'soil_depth_statsgo', 'soil_porosity','soil_conductivity',
                'max_water_content', 'sand_frac', 'silt_frac', 'clay_frac',
                #'water_frac', 'organic_frac', 'other_frac',
                #'gauge_lat','gauge_lon',
                'elev_mean', 'slope_mean',
                #'area_gages2',
                'area_geospa_fabric',
                'frac_forest', 'lai_max', 'lai_diff', 'gvf_max','gvf_diff',
                #'dom_land_cover_frac', 'dom_land_cover', 'root_depth_50','root_depth_99', 
#                                  'hru08'
                ]]

In [5]:
selected_basins = pd.read_csv('/Users/omernivron/Downloads/basin_list.txt', header=None)

In [6]:
df = pd.read_csv('/Users/omernivron/Downloads/daymet_data_seed05.csv')

In [7]:
context_channels = ['OBS_RUN',
                    'doy_cos','doy_sin',
                    'prcp(mm/day)', 
                    'srad(W/m2)',  
                    'tmax(C)',
                    'tmin(C)', 
                    'vp(Pa)'] 

In [40]:
context_channels2 = ['prcp(mm/day)', 
                    'srad(W/m2)',  
                    'tmax(C)',
                    'tmin(C)', 
                    'vp(Pa)'] 

In [8]:
list_to_drop = ['MNTH', 'DY', 'hru02', 'hru04', 'RAIM', 'TAIR', 'PET', 'ET', 'SWE', 'swe(mm)', 'PRCP', 'seed', 'id_lag', 'HR', 'dayl(s)', 'YR', 'MOD_RUN', 'id', 'DOY', 'DATE']

In [9]:
df.drop(columns= list_to_drop, inplace=True)
df.drop_duplicates(inplace=True)
df = df[df['OBS_RUN'] >= 0]
cols = df.columns.to_list()
cols = [cols[5]] + cols[:5] + cols[6:]
df = df[cols]

In [11]:
arr_tr = np.zeros((200 * len(selected_basins), 50, 9))
arr_te = np.zeros((200 * (len(df.basin.unique()) - len(selected_basins)), 50, 9))

In [36]:
step_tr = 0
step_te = 0
for bas in df.basin.unique():
    df_temp = (df[df['basin'] == bas]).reset_index()
    att_num = np.where(attributes_numeric['hru08'] == df_temp['hru08'].unique()[0])[0]
    rep_att_num = np.repeat(att_num, 50).reshape(-1, 1)
    for obs in range(200):
        rows = np.random.choice(np.arange(0, df_temp.shape[0], 1), 50)
        if (np.isin(bas, selected_basins)):
            arr_tr[step_tr, :, :] = np.concatenate((df_temp.loc[rows, context_channels], rep_att_num), axis = 1)
            step_tr += 1
        else:
            arr_te[step_te, :, :] = np.concatenate((df_temp.loc[rows, context_channels], rep_att_num), axis = 1)
            step_te += 1

In [37]:
np.save('/Users/omernivron/Downloads/river_flow_tr', arr_tr)
np.save('/Users/omernivron/Downloads/river_flow_te', arr_te)

In [46]:
arr_pp = np.zeros((5 * 10000, 50))

In [70]:
for i in range(len(context_channels2)):
    for row in range(0 + 10000 * i, 10000 * (i + 1), 5):
        basin = np.random.choice(selected_basins[0], 2)
        df_temp = (df[df['basin'] == basin[0]]).reset_index()
        df_riv = (df[df['basin'] == basin[1]]).reset_index()

        idx_i = np.random.choice(np.arange(0, df_temp.shape[0], 1), 25)
        idx_riv = np.random.choice(np.arange(0, df_riv.shape[0], 1), 25)
        
        arr_pp[row, :] = np.concatenate((df_temp.loc[idx_i, [context_channels2[i]]], df_riv.loc[idx_riv, ['OBS_RUN']]), axis = 0).reshape(-1)
        arr_pp[row + 1, :] = np.concatenate((df_temp.loc[idx_i, ['doy_cos']], df_riv.loc[idx_riv, ['doy_cos']]), axis = 0).reshape(-1)
        arr_pp[row + 2, :] = np.concatenate((df_temp.loc[idx_i, ['doy_sin']], df_riv.loc[idx_riv, ['doy_sin']]), axis = 0).reshape(-1)
        arr_pp[row + 3, :] = np.concatenate((np.ones(25) * i, np.ones(25) * 9))
        arr_pp[row + 4, :] = np.concatenate((df_temp.loc[idx_i, ['basin']], df_riv.loc[idx_riv, ['basin']]), axis = 0).reshape(-1)

In [125]:
np.save('/Users/omernivron/Downloads/river_processed', arr_pp)

In [5]:
arr_pp = np.load('/Users/omernivron/Downloads/river_processed.npy')

In [6]:
save_dir = '/Users/omernivron/Downloads/GPT_river'

In [14]:
loss_object = tf.keras.losses.MeanSquaredError()
train_loss = tf.keras.metrics.Mean(name='train_loss')
test_loss = tf.keras.metrics.Mean(name='test_loss')
m_tr = tf.keras.metrics.Mean()
m_te = tf.keras.metrics.Mean()

In [15]:
@tf.function
def train_step(decoder, optimizer_c, train_loss, m_tr, token_pos, time_pos, time_pos2, pos, tar, pos_mask):
    '''
    A typical train step function for TF2. Elements which we wish to track their gradient
    has to be inside the GradientTape() clause. see (1) https://www.tensorflow.org/guide/migrate 
    (2) https://www.tensorflow.org/tutorials/quickstart/advanced
    ------------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask (np array): see description in position_mask function
    ------------------    
    '''
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]
    combined_mask_tar = masks.create_masks(tar_inp)
    with tf.GradientTape(persistent=True) as tape:
        pred, pred_sig = decoder(token_pos, time_pos, time_pos2, pos, tar_inp, True, pos_mask, combined_mask_tar)
#         print('pred: ')
#         tf.print(pred_sig)

        loss, mse, mask = losses.loss_function(tar_real, pred, pred_sig)


    gradients = tape.gradient(loss, decoder.trainable_variables)
#     tf.print(gradients)
# Ask the optimizer to apply the processed gradients.
    optimizer_c.apply_gradients(zip(gradients, decoder.trainable_variables))
    train_loss(loss)
    m_tr.update_state(mse, mask)
#     b = decoder.trainable_weights[0]
#     tf.print(tf.reduce_mean(b))
    return tar_inp, tar_real, pred, pred_sig, mask

In [16]:
@tf.function
def test_step(decoder, test_loss, m_te, token_pos_te, time_pos_te, pos_te, tar_te, pos_mask_te):
    '''
    
    ---------------
    Parameters:
    pos (np array): array of positions (x values) - the 1st/2nd output from data_generator_for_gp_mimick_gpt
    tar (np array): array of targets. Notice that if dealing with sequnces, we typically want to have the targets go from 0 to n-1. The 3rd/4th output from data_generator_for_gp_mimick_gpt  
    pos_mask_te (np array): see description in position_mask function
    ---------------
    
    '''
    tar_inp_te = tar_te[:, :-1]
    tar_real_te = tar_te[:, 1:]
    combined_mask_tar_te = masks.create_masks(tar_inp_te)
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
    pred_te, pred_sig_te = decoder(token_pos_te, time_pos_te, tar_inp_te, False, pos_mask_te, combined_mask_tar_te)
    t_loss, t_mse, t_mask = losses.loss_function(tar_real_te, pred_te, pred_sig_te)
    test_loss(t_loss)
    m_te.update_state(t_mse, t_mask)
    return tar_real_te, pred_te, pred_sig_te, t_mask

In [17]:
tf.keras.backend.set_floatx('float64')

In [18]:
t1 = arr_pp[1::5]; t2 = arr_pp[2::5]
tar = arr_pp[0::5];
token = arr_pp[3::5]; basin_l = arr_pp[4::5] 

In [ ]:
if __name__ == '__main__':
    writer = tf.summary.create_file_writer(save_dir + '/logs/')
    optimizer_c = tf.keras.optimizers.Adam()
    decoder = river_model.Decoder(16)
    EPOCHS = 500
    batch_s  = 32
    run = 0; step = 0
    num_batches = int(tar.shape[0] / batch_s)
    tf.random.set_seed(1)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer = optimizer_c, net = decoder)
    main_folder = "/Users/omernivron/Downloads/GPT_river/ckpt/check_"
    folder = main_folder + str(run); helpers.mkdir(folder)
    #https://www.tensorflow.org/guide/checkpoint
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=3)
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print("Restored from {}".format(manager.latest_checkpoint))
    else:
        print("Initializing from scratch.")

    with writer.as_default():
        for epoch in range(EPOCHS):
            start = time.time()

            for batch_n in range(num_batches):
                m_tr.reset_states(); train_loss.reset_states()
                m_te.reset_states(); test_loss.reset_states()
                batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, _ = batch_creator.create_batch_river(token, t1, t2, basin_l,  attributes_numeric, tar, batch_s=32)
                # batch_tar_tr shape := 128 X 59 = (batch_size, max_seq_len)
                # batch_pos_tr shape := 128 X 59 = (batch_size, max_seq_len)
                batch_pos_mask = masks.position_mask(batch_tok_pos_tr)
                tar_inp, tar_real, pred, pred_sig, mask = train_step(decoder, optimizer_c, train_loss, m_tr, batch_tok_pos_tr, batch_tim_pos_tr, batch_tim_pos_tr2, batch_pos_tr, batch_tar_tr, batch_pos_mask)

#                 if batch_n % 10000 == 0:
#                     batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, _ = batch_creator.create_batch_river(token, t1, t2, basin_l,  att, tar, batch_s=32)
#                     batch_pos_mask_te = masks.position_mask(batch_tok_pos_te)
#                     tar_real_te, pred_te, pred_sig_te, t_mask = test_step(batch_tok_pos_te, batch_tim_pos_te, batch_tar_te, batch_pos_mask_te)
#                     helpers.print_progress(epoch, batch_n, train_loss.result(), test_loss.result(), m_tr.result())
#                     helpers.tf_summaries(run, step, train_loss.result(), test_loss.result(), m_tr.result(), m_te.result())
#                     manager.save()
                step += 1
                ckpt.step.assign_add(1)

            print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Already exists
Initializing from scratch.
Tensor("decoder_2/embedding_2/embedding_lookup/Identity:0", shape=(32, 50, 8), dtype=float64)
Tensor("decoder_2/concat:0", shape=(32, 50, 9), dtype=float64)
Tensor("decoder_2/embedding_2/embedding_lookup/Identity:0", shape=(32, 50, 8), dtype=float64)
Tensor("decoder_2/concat:0", shape=(32, 50, 9), dtype=float64)


In [13]:
attributes_numeric

,gauge_id,p_mean,pet_mean,p_seasonality,frac_snow,aridity,high_prec_freq,high_prec_dur,low_prec_freq,low_prec_dur,...,silt_frac,clay_frac,elev_mean,slope_mean,area_geospa_fabric,frac_forest,lai_max,lai_diff,gvf_max,gvf_diff
0,1013500,3.126679,1.971555,0.187940,0.313440,0.630559,12.95,1.348958,202.20,3.427119,...,55.156940,16.275732,250.31,21.64152,2303.95,0.9063,4.167304,3.340732,0.804567,0.371648
1,1022500,3.608126,2.119256,-0.114530,0.245259,0.587356,20.55,1.205279,233.65,3.662226,...,28.080937,12.037646,92.68,17.79072,620.38,0.9232,4.871392,3.746692,0.863936,0.337712
2,1030500,3.274405,2.043594,0.047358,0.277018,0.624111,17.15,1.207746,215.60,3.514262,...,51.779182,14.776824,143.80,12.79195,3676.09,0.8782,4.685200,3.665543,0.858502,0.351393
3,1031500,3.522957,2.071324,0.104091,0.291836,0.587950,18.90,1.148936,227.35,3.473644,...,50.841232,12.654125,247.80,29.56035,766.53,0.9548,4.903259,3.990843,0.870668,0.398619
4,1047000,3.323146,2.090024,0.147776,0.280118,0.628929,20.10,1.165217,235.90,3.691706,...,34.185443,10.303622,310.38,49.92122,904.94,0.9906,5.086811,4.300978,0.891383,0.445473
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
666,14309500,4.977781,3.122204,-0.995847,0.061255,0.627228,15.10,1.776471,222.65,6.893189,...,38.879406,23.213862,709.83,110.42527,226.31,1.0000,4.227902,1.986325,0.883414,0.115741
667,14316700,4.543400,2.277630,-0.821172,0.176337,0.501305,14.75,1.446078,214.85,6.018207,...,38.519396,24.363634,952.26,119.08920,588.01,1.0000,4.859652,2.828735,0.914354,0.171176
668,14325000,6.297437,2.434652,-0.952055,0.030203,0.386610,14.60,1.467337,219.05,6.240741,...,40.860260,20.068726,656.53,124.96889,444.92,1.0000,4.150730,1.867148,0.873517,0.115977
669,14362250,2.781676,3.325188,-0.985486,0.141500,1.195390,20.45,1.786026,260.35,7.354520,...,39.602460,22.404372,875.67,109.93127,43.88,1.0000,4.430338,2.451489,0.868294,0.117102


In [17]:
xx[i, 0]

1586610.0

In [19]:
np.array(attributes.iloc[np.where([attributes['gauge_id'] == xx[i, 0]])[1][0], 1:])

array([3.30466119096509, 2.54404796714579, 0.08026925136315599,
       0.088413478566147, 0.769836246481542, 22.4, 1.20107238605898,
       'son', 249.1, 4.2948275862069, 'son', 'Metamorphics',
       0.961900930450197, 'Basic volcanic rocks', 0.0380990695498032, 0.0,
       0.013, -14.039, 1.16319464933894, 0.35198605306925296,
       1.49041360174604, 0.661737774386697, 1.22591311720456,
       0.235571235472374, 3.02877302750195, 3.5, 1.12903225806452, 18.1,
       9.28205128205128, 0.0, 168.5, 2, 'MORGAN RUN NEAR LOUISVILLE, MD',
       1.01680672268908, 1.35970209358477, 0.455757721489515,
       1.16555746012633, 0.576391109817708, 29.0265605450293,
       49.23108413474521, 16.0880342853558, 0.0, 0.0, 5.611916852655572,
       39.45189, -76.95531, 196.88, 11.85286, 72.7, 72.81, 0.3247,
       2.51629324989542, 2.0915919074814, 0.7268470895212249,
       0.40583064071036296, 1.0, '    cropland/natural vegetation mosaic',
       0.18, 1.5, 'hru_01586610'], dtype=object)

In [20]:
batch_pos_tr = np.zeros((32, attributes_numeric.shape[1] - 1, tar.shape[1]))
xx = basin_l[:32, :]
for i in range(xx.shape[0]):
    batch_pos_tr[i, :, :] = np.concatenate(((np.repeat(np.array(attributes_numeric.iloc[np.where
                                                            ([attributes_numeric['gauge_id'] == xx[i, 0]])[1][0], 1:]).reshape(1, -1), 25)).reshape(-1, 25), (np.repeat(np.array(attributes_numeric.iloc[np.where
                                                                                                                                                                                                        ([attributes_numeric['gauge_id'] == xx[i, -1]])[1][0], 1:]).reshape(1, -1), 25)).reshape(-1, 25)), axis=1)